SPARK 18 - Calcule el tamaño promedio de las aplicaciones por versión de Android, sin tener en cuenta las aplicaciones que varían en tamaño según dispositivo. (⭐⭐)

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [19]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import math
import re

In [4]:
# Crea la sesion de SPARK
spark = SparkSession.builder.getOrCreate()

# Crea el SPARK content
sc = spark.sparkContext

In [5]:
sqlContext = SQLContext(sc)
infoplay = sqlContext.read.csv("/content/drive/MyDrive/Colab Notebooks/TP1-VISU/googleplaystore.csv", header=True)
inforeviews = sqlContext.read.csv("/content/drive/MyDrive/Colab Notebooks/TP1-VISU/googleplaystore_user_reviews.csv", header=True)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [20]:
infoplayRDD = infoplay.rdd

Se filtra el RDD para eliminar registros no deseados

In [21]:
filtro = infoplayRDD.filter(lambda x: x['Size'] != 'Varies with device' and all(
    elementos is not None for elementos in x) and x['Android Ver'] != 'Varies with device')

Se realizan transformaciones en los registros filtrados

In [22]:
def procesar_valor(texto):
    parte_numerica = re.search(r'\d+(\.\d+)?', texto)
    if parte_numerica:
        valor_numerico = float(parte_numerica.group())
        if 'M' in texto:
            valor_numerico *= 1000000
        elif 'k' in texto:
            valor_numerico *= 1000
        return valor_numerico
    else:
        return None

In [23]:
transformacion = filtro.map(lambda x: (
    x['Android Ver'].replace('and up', '').replace('W', ''), (procesar_valor(x['Size']), 1)))

Modifico todas las claves que son de la forma ('5.0 - 7.1.1') para que se conviertan en dos claves distintas como '5.0' y '7.11'

In [24]:
separacion_claves = transformacion.flatMap(lambda x: [(componente.strip(), x[1]) for componente in x[0].split('-')])

Hago un reduce por cada clave, sumando los valores numéricos y de conteo

In [25]:
reduccion_claves = separacion_claves.reduceByKey(lambda x, y: (int(x[0]) + int(y[0]), x[1] + y[1]))

Filtro las claves que sobrevivieron al filtrado anterior y que pueden llegar a ocasionar errores como la version de andriod 'Maps & Navigation'

In [26]:
def laClaveEsCorrecta(cadena):
    if ((cadena == 'Maps & Navigation') or (cadena == 'Health & Fitness') or (cadena == 'NaN')):
      return False
    else :
      return True

In [27]:
filtro2 = reduccion_claves.filter(lambda x: laClaveEsCorrecta(x[0]))

Realiza cálculos adicionales y redondea el resultado. La division de 10000000 es porque estamos pasando de Bytes a MegaBytes

In [28]:
calculos_finales = filtro2.map(lambda x: (
    x[0], (math.ceil(int(x[1][0]) / int(x[1][1])) / 1000000)))

Se muestran la respectivas versiones de andriod y el promedio de espacio que ocupan las aplicaciones de esa version. El temaño esta en MegaBytes (MB)

In [29]:
calculos_finales.collect()

[('4.0.3', 22.060229),
 ('4.2', 22.140095),
 ('4.4', 25.653754),
 ('2.3', 20.809875),
 ('3.0', 14.699756),
 ('4.1', 26.02796),
 ('4.0', 17.104884),
 ('2.2', 8.485711),
 ('5.0', 24.609808),
 ('6.0', 22.309207),
 ('1.6', 3.119138),
 ('2.1', 5.605463),
 ('5.1', 23.622305),
 ('1.5', 5.352778),
 ('7.0', 23.894781),
 ('4.3', 22.60336),
 ('7.1.1', 6.95),
 ('2.0', 6.256807),
 ('2.3.3', 19.221511),
 ('3.2', 9.860639),
 ('7.1', 32.00425),
 ('8.0', 21.0125),
 ('3.1', 13.4138),
 ('2.0.1', 22.222143),
 ('1.0', 3.8555)]